In [ ]:
import os
if not os.path.exists('data'):
    !git clone https://github.com/iceghost/job-shop-genetic-algorithm
    %cd job-shop-genetic-algorithm

In [206]:
import pandas as pd
import numpy as np
import random as rand

input = pd.read_json("data/input.json")
machines = pd.read_json("data/machines.json")


In [207]:
input


,id,component,amount,sequence,deadline
0,2264WP4021,W50P37F13,1703,"[4, 3, 6, 5, 9, 10]",57600
1,2233RU2124(2),P69R29PU2,1500,"[1, 2, 4, 5, 3, 6, 8]",57600
2,"2233PT2134(1),(2)",W50P50,5565,"[3, 5, 0, 4, 6, 9, 8, 7]",59040
3,2247PU3040(3),W30P70,2000,"[3, 5, 0, 4, 6, 8, 9]",60480
4,2247PU3031(5),W30P70,2500,"[3, 5, 0, 4, 6, 9]",61920
5,2239WP2140,W51P43PU6,5568,"[0, 2, 4, 9, 5, 3, 6, 7, 8]",63360
6,DUC93324S1W2#10B,W50P37F13,11000,"[3, 5, 0, 4, 6, 7, 8, 10]",63360
7,2233RU2101,P69R29PU2,500,"[4, 5, 0, 6, 3, 7, 8, 10]",63360
8,2160PU4006(4),W50P50,9370,"[3, 5, 0, 4, 6, 8, 7]",64800
9,2160PU4005(1)(2),W50P50,15330,"[3, 5, 0, 4, 6, 9, 8, 7]",64800


In [208]:
machines


,id,name,speed,set_up_time
0,R/S,Flexirapid 600,20,20
1,M/S,Twin 800,10,20
2,R/O,Corino,6,15
3,W,Aqua,20,15
4,ST,Brueckner,42,15
5,SI,Osthoff,60,10
6,CM,None,20,1
7,D,Thermoduplex,18,15
8,KD,Corino,25,20
9,P,Formula 1,17,20


## Khởi tạo các ma trận

Khởi tạo ma trận tuần tự công nghệ, `-1` là không có.

In [209]:
Tjk = pd.DataFrame(input["sequence"].values.tolist()).fillna(-1).astype(int).values
Tjk


array([[ 4,  3,  6,  5,  9, 10, -1, -1, -1],
       [ 1,  2,  4,  5,  3,  6,  8, -1, -1],
       [ 3,  5,  0,  4,  6,  9,  8,  7, -1],
       [ 3,  5,  0,  4,  6,  8,  9, -1, -1],
       [ 3,  5,  0,  4,  6,  9, -1, -1, -1],
       [ 0,  2,  4,  9,  5,  3,  6,  7,  8],
       [ 3,  5,  0,  4,  6,  7,  8, 10, -1],
       [ 4,  5,  0,  6,  3,  7,  8, 10, -1],
       [ 3,  5,  0,  4,  6,  8,  7, -1, -1],
       [ 3,  5,  0,  4,  6,  9,  8,  7, -1]])

In [210]:
Cij = np.zeros((len(machines), len(input)), dtype=int)
for j, i in enumerate(Tjk[:, 0]):
    if i != -1:
        Cij[i, j] = 1
Cij


array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 1, 0, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [211]:
L_Tij = np.ndarray((len(machines), len(input)), dtype=int)

for i in range(0, len(machines)):
    for j in range(0, len(input)):
        try:
            # máy tiếp theo của máy i là máy gì?
            L_Tij[i, j] = Tjk[j, np.where(Tjk[j] == i)[0][0] + 1]

        except IndexError:
            # nếu không có máy tiếp theo thì gán -1
            L_Tij[i, j] = -1

L_Tij


array([[-1, -1,  4,  4,  4,  2,  4,  6,  4,  4],
       [-1,  2, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  4, -1, -1, -1,  4, -1, -1, -1, -1],
       [ 6,  6,  5,  5,  5,  6,  5,  7,  5,  5],
       [ 3,  5,  6,  6,  6,  9,  6,  5,  6,  6],
       [ 9,  3,  0,  0,  0,  3,  0,  0,  0,  0],
       [ 5,  8,  9,  8,  9,  7,  7,  3,  8,  9],
       [-1, -1, -1, -1, -1,  8,  8,  8, -1, -1],
       [-1, -1,  7,  9, -1, -1, 10, 10,  7,  7],
       [10, -1,  8, -1, -1,  5, -1, -1, -1,  8],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [212]:
F_Tij = np.ndarray((len(machines), len(input)), dtype=int)

for i in range(0, len(machines)):
    for j in range(0, len(input)):
        try:
            # máy tiếp theo của máy i là máy gì?
            next_k = np.where(Tjk[j] == i)[0][0] - 1

            if next_k == -1:
                raise IndexError()

            F_Tij[i, j] = Tjk[j, next_k]

        except IndexError:
            # nếu không có máy tiếp theo thì gán -1
            F_Tij[i, j] = -1

F_Tij


array([[-1, -1,  5,  5,  5, -1,  5,  5,  5,  5],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  1, -1, -1, -1,  0, -1, -1, -1, -1],
       [ 4,  5, -1, -1, -1,  5, -1,  6, -1, -1],
       [-1,  2,  0,  0,  0,  2,  0, -1,  0,  0],
       [ 6,  4,  3,  3,  3,  9,  3,  4,  3,  3],
       [ 3,  3,  4,  4,  4,  3,  4,  0,  4,  4],
       [-1, -1,  8, -1, -1,  6,  6,  3,  8,  8],
       [-1,  6,  9,  6, -1,  7,  7,  7,  6,  9],
       [ 5, -1,  6,  8,  6,  4, -1, -1, -1,  6],
       [ 9, -1, -1, -1, -1, -1,  8,  8, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [213]:
T_Oij = np.ndarray((len(machines), len(input)))
for i in range(0, len(machines)):
    for j in range(0, len(input)):
        T_Oij[i, j] = input.loc[j]["amount"] / machines.loc[i]["speed"]
T_Oij


array([[  85.15      ,   75.        ,  278.25      ,  100.        ,
         125.        ,  278.4       ,  550.        ,   25.        ,
         468.5       ,  766.5       ],
       [ 170.3       ,  150.        ,  556.5       ,  200.        ,
         250.        ,  556.8       , 1100.        ,   50.        ,
         937.        , 1533.        ],
       [ 283.83333333,  250.        ,  927.5       ,  333.33333333,
         416.66666667,  928.        , 1833.33333333,   83.33333333,
        1561.66666667, 2555.        ],
       [  85.15      ,   75.        ,  278.25      ,  100.        ,
         125.        ,  278.4       ,  550.        ,   25.        ,
         468.5       ,  766.5       ],
       [  40.54761905,   35.71428571,  132.5       ,   47.61904762,
          59.52380952,  132.57142857,  261.9047619 ,   11.9047619 ,
         223.0952381 ,  365.        ],
       [  28.38333333,   25.        ,   92.75      ,   33.33333333,
          41.66666667,   92.8       ,  183.33333333,    8

## Khởi tạo quần thể ban đầu

In [214]:
P = np.ndarray((len(machines), len(input)), dtype=int)
P.fill(-1)
G = set(tuple(ij) for ij in np.column_stack(np.where(Cij == 1)))
T_Sij = np.zeros((len(machines), len(input)))
T_Fij = T_Sij + T_Oij
VTi = np.zeros(len(machines), dtype=int)

while True:
    # V_G chứa các cặp (i, j) mà T_Sij[i, j] nhỏ nhất
    V_G = set(
        (i, j)
        for (i, j) in G
        if T_Sij[i, j] == T_Sij[list(g[0] for g in G), list(g[1] for g in G)].min()
    )

    # xóa các cặp (i, j) không có trong G
    V_G.intersection_update(G)

    # chọn ngẫu nhiên từ V_G
    i, j = rand.choice(list(V_G))

    P[i, VTi[i]] = j
    VTi[i] += 1
    G.remove((i, j))

    # nếu k chưa phải là máy cuối cùng thì thêm vào G
    if (k := L_Tij[i, j]) != -1:
        G.add((k, j))

    # nếu điền xong hết rồi thì thoát ra
    elif len(G) == 0:
        break

    # cập nhật lại T_Sij, T_Fij
    for i, j in G:
        r = P[i, VTi[i] - 1]
        k = F_Tij[i, j]
        T_Sij[i, j] = max(T_Fij[i, r], T_Fij[k, j])
        T_Fij[i, j] = T_Sij[i, j] + T_Oij[i, j]
P


array([[ 5,  8,  9,  7,  4,  6,  3,  2, -1, -1],
       [ 1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 1,  5, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 9,  3,  2,  0,  4,  8,  6,  5,  1,  7],
       [ 0,  7,  1,  5,  8,  9,  4,  6,  3,  2],
       [ 5,  8,  9,  2,  1,  6,  0,  3,  4,  7],
       [ 0,  5,  1,  8,  7,  9,  3,  4,  6,  2],
       [ 5,  7,  8,  6,  9,  2, -1, -1, -1, -1],
       [ 1,  5,  8,  7,  3,  9,  2,  6, -1, -1],
       [ 5,  0,  9,  4,  3,  2, -1, -1, -1, -1],
       [ 0,  7,  6, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [215]:
T_Fij


array([[   85.15      ,    75.        ,  7416.71666667,  7138.46666667,
         6488.46666667,  1044.9       ,  7038.46666667,  6363.46666667,
         5571.96666667,  6338.46666667],
       [  170.3       ,  1683.        ,   556.5       ,   200.        ,
          250.        ,   556.8       ,  1100.        ,    50.        ,
          937.        ,  1533.        ],
       [  283.83333333,  2805.        ,   927.5       ,   333.33333333,
          416.66666667,  3733.        ,  1833.33333333,    83.33333333,
         1561.66666667,  2555.        ],
       [ 1229.9       ,  5551.71666667,  1144.75      ,   866.5       ,
         1354.9       ,  5225.7       ,  2373.4       ,  6413.46666667,
         1823.4       ,   766.5       ],
       [  405.54761905,  2840.71428571,  7549.21666667,  7347.99047619,
         6762.99047619,  3865.57142857,  7300.37142857,   417.45238095,
         5795.06190476,  6703.46666667],
       [ 5688.43333333,  5476.71666667,  5451.71666667,  5721.76666667,
   

In [216]:
T_FJj = np.max(T_Fij, axis=0)
T_FJj


array([ 5930.52647059,  5686.71666667, 10145.76470588,  8636.4372549 ,
        8518.79019608,  6036.15333333, 10564.19803922,  6700.02857143,
        7158.91746032,  9836.59803922])

In [217]:
input["deadline"].values


array([57600, 57600, 59040, 60480, 61920, 63360, 63360, 63360, 64800,
       64800], dtype=int64)

In [218]:
T_TJj = T_FJj - input["deadline"].values
T_TJj = T_TJj.clip(min=0)
T_TJj


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])